In [5]:
import numpy as np
import sys
import math
import os
import torch
from tqdm import tqdm
from tensorboardX import SummaryWriter

import copy

# from models import LinearModel, convert_vars_to_gpu
from utils import logsumexp, parse_my_args_reinforce, shuffle_combined, torchify, exp_lr_scheduler, get_optimizer

# from models import LinearModel
from torch.autograd import Variable
from datareader import reader_from_pickle

In [9]:
import torch
from torch import nn
from torch.autograd import Variable

class LinearModel(nn.Module):
    """
    One layer simple linear model
    """
    def __init__(self, D=2, clamp=False):
        self.input_dim = D
        super(LinearModel, self).__init__()
        self.w = nn.Linear(D, 1, bias=True)
        self.w.weight.data.uniform_(-0.0000001, 0.0000001)
        self.clamp = clamp

    def forward(self, x):
        h = self.w(x)
        return h if not self.clamp else torch.clamp(h, -10, 10)

In [10]:
data_reader, validation_data_reader = reader_from_pickle(
    "GermanCredit/partial_german_train_rank_weightedclick_10k_old.pkl"), reader_from_pickle(
    "GermanCredit/partial_german_test_rank_weightedclick_10k_old.pkl")

train_feats, train_rel = data_reader.data
train_feats, train_rel = torch.FloatTensor(train_feats), torch.FloatTensor(train_rel)
train_feats, train_rel = shuffle_combined(train_feats, train_rel)

model = LinearModel(D=29)

feats,rels = train_feats[530], train_rel[530]
scores = model(Variable(feats))
print(scores)
probs = torch.nn.Softmax(dim=0)(scores).flatten()
print(probs)

tensor([[0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364],
        [0.0364]], grad_fn=<ThAddmmBackward>)
tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], grad_fn=<AsStridedBackward>)


In [11]:
model = LinearModel(D=58)
data_reader, validation_data_reader = reader_from_pickle(
    "GermanCredit/partial_german_train_rank_weightedclick_10k.pkl"), reader_from_pickle(
    "GermanCredit/partial_german_test_rank_weightedclick_10k.pkl")
train_feats, train_rel = data_reader.data
train_feats, train_rel = torch.FloatTensor(train_feats), torch.FloatTensor(train_rel)
train_feats, train_rel = shuffle_combined(train_feats, train_rel)
feats,rels = train_feats[0], train_rel[0]
scores = model(Variable(feats))
print("scores", scores)
scores_ = scores/scores.sum()
print("scores_", scores_)
probs = torch.nn.Softmax(dim=0)(scores).flatten()
print("probs", probs)
probs_ = torch.nn.Softmax(dim=0)(scores_).flatten()
print("probs_", probs_)
print(feats.shape, rels.shape)

scores tensor([[0.1189],
        [0.1189],
        [0.1189],
        [0.1189],
        [0.1188],
        [0.1189],
        [0.1190],
        [0.1188],
        [0.1186],
        [0.1188]], grad_fn=<ThAddmmBackward>)
scores_ tensor([[0.1001],
        [0.1000],
        [0.1001],
        [0.1000],
        [0.1000],
        [0.1001],
        [0.1001],
        [0.1000],
        [0.0998],
        [0.1000]], grad_fn=<DivBackward1>)
probs tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], grad_fn=<AsStridedBackward>)
probs_ tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], grad_fn=<AsStridedBackward>)
torch.Size([10, 58]) torch.Size([10])


In [ ]:
from evaluation import compute_dcg, evaluate_model, sample_ranking, compute_average_rank

In [ ]:
rankings, rewards_list, ndcg_list, dcg_list = [], [], [], []
for j in range(5):
    ranking = sample_ranking(probs, False)
    rankings.append(ranking)

In [ ]:
# rels = torch.tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
rankings, rels

In [ ]:
group_identities = feats[:, 3]
t_rankings = torch.stack(rankings)
rankings, skip_zero,rel = t_rankings, True, rels

In [ ]:
from evaluation import get_exposures
position_bias_vector = 1./torch.arange(1.,11.)
get_exposures(ranking, position_bias_vector)

In [ ]:
num_docs =10
position_bias_vector[:num_docs].unsqueeze(0).repeat(rankings.shape[0],1)
position_biases = position_bias_vector

In [ ]:
exposures = torch.stack([get_exposures(ranking, position_biases) for i in rankings])
inds_g0 = group_identities == 0
inds_g1 = group_identities == 1
if skip_zero:
    inds_g0 = inds_g0 * (rel != 0)
    inds_g1 = inds_g1 * (rel != 0)


In [ ]:
inds_g0 * (rel != 0)

In [ ]:
def get_exposures(ranking, position_bias_vector):
    num_docs = len(ranking)
    exposure = np.zeros(num_docs)
    exposure[ranking] = position_bias_vector[:num_docs]
    return exposure

def compute_group_disparity(ranking,
                            rel,
                            group_identities,
                            position_biases,
                            skip_zero=False):
    exposures = get_exposures(ranking, position_biases)
    inds_g0 = group_identities == 0
    inds_g1 = group_identities == 1
    if skip_zero:
        inds_g0 = np.logical_and(inds_g0, rel != 0)
        inds_g1 = np.logical_and(inds_g1, rel != 0)
    return np.sum(exposures[inds_g0]) / np.sum(rel[inds_g0]) - np.sum(
        exposures[inds_g1]) / np.sum(rel[inds_g1])

In [ ]:
compute_group_disparity(rankings[4].numpy(), rels.numpy(), group_identities.numpy(), position_bias_vector.numpy())

In [ ]:
def get_exposures(ranking, position_bias_vector):
    num_docs = len(ranking)
    exposure = torch.zeros(num_docs)
    exposure[ranking] = position_bias_vector[:num_docs]
    return exposure

def compute_group_disparity(ranking,
                            rel,
                            group_identities,
                            position_biases,
                            skip_zero=False):
    exposures = get_exposures(ranking, position_biases)
    inds_g0 = group_identities == 0
    inds_g1 = group_identities == 1
    if skip_zero:
        inds_g0 = inds_g0 * (rel != 0)
        inds_g1 = inds_g1 * (rel != 0)
    return torch.sum(exposures[inds_g0]) / torch.sum(rel[inds_g0]) - torch.sum(
        exposures[inds_g1]) / torch.sum(rel[inds_g1])

In [ ]:
compute_group_disparity(rankings[0], rels, group_identities, position_bias_vector)

In [ ]:
rankings_, rels_, group_identities_, position_bias_vector_ = rankings[4].numpy(), rels.numpy(), group_identities.numpy(), position_bias_vector.numpy()


In [ ]:
from train import compute_log_model_probability
from utils import logsumexp

In [ ]:
compute_log_model_probability(scores, rankings[3])

In [ ]:
subtracts = torch.zeros_like(scores)
log_probs = torch.zeros_like(scores)
# if gpu:
#     subtracts, log_probs = convert_vars_to_gpu([subtracts, log_probs])
for j in range(scores.size()[0]):
    posj = ranking[j]
    log_probs[j] = scores[posj] - logsumexp(scores - subtracts, dim=0)
    subtracts[posj] = scores[posj] + 1e6
torch.sum(log_probs)
# log_probs

In [ ]:
#scores, ranking
print(probs)
log_probs = torch.zeros_like(probs)
for posj in ranking:
    log_probs[posj] = torch.log(probs)[posj]
    probs[posj] = 0
    probs = torch.nn.functional.softmax(probs)
    print(probs)
torch.sum(log_probs)

In [ ]:
#scores, ranking
print(scores)
scores_= scores
log_probs = torch.zeros_like(scores_)
print(torch.nn.functional.log_softmax(scores_, dim=0))
for posj in ranking:
    log_probs[posj] = torch.nn.functional.log_softmax(scores_, dim=0)[posj]
    scores_[posj] = -1e6
torch.sum(log_probs)

In [ ]:
subtracts = torch.zeros_like(scores)
log_probs = torch.zeros_like(scores)
# subtracts[ranking] = scores[ranking] + 1e6
for j in range(len(ranking)):
    log_probs[j] = scores[j] - logsumexp(scores - subtracts, dim=0)
    subtracts[j] = scores[j] + 1e6
#     print(logsumexp(scores - subtracts, dim=0))
# print(log_probs, ranking)
torch.sum(log_probs)
log_probs

In [ ]:
torch.nn.functional.log_softmax(scores,dim=0)

In [ ]:
subtracts = torch.zeros_like(scores)
log_probs = torch.zeros_like(scores)
# if gpu:
#     subtracts, log_probs = convert_vars_to_gpu([subtracts, log_probs])
# for j in range(scores.size()[0]):
j=torch.arange(scores.size()[0])
posj = ranking[j]
subtracts[posj] = scores[posj] + 1e6
log_probs[j] = scores[posj] - logsumexp(scores - subtracts, dim=0)
torch.sum(log_probs)

In [ ]:
import matplotlib.pyplot as plt
# plt.ion()

In [ ]:
rewards = [0.6770715117454529,
0.6976408958435059,
0.7047403454780579,
0.7139430642127991,
0.7054921388626099,
0.6972674131393433,
0.7208537459373474,
0.71114182472229,
0.7081214785575867,
0.6965669393539429]
ndcgs = [0.6770715117454529,
0.6976408958435059,
0.7047403454780579,
0.7139430642127991,
0.7054921388626099,
0.6972674131393433,
0.7208537459373474,
0.71114182472229,
0.7081214785575867,
0.6965669393539429]
dcgs = [47.724464416503906,
53.05571365356445,
56.91681671142578,
58.481204986572266,
56.369693756103516,
56.18364715576172,
51.54362106323242,
62.39633560180664,
61.6878662109375,
57.674381256103516]

In [ ]:
torch.sort(-probs)[1]

In [12]:
.00001

1e-05

In [17]:
5e-6 == .000005

True

In [19]:
float("1e-5")+1

1.00001